# Rodent baiting requests and rat complaints made to the service
![baiting](https://upload.wikimedia.org/wikipedia/commons/thumb/e/eb/Das_festliche_Jahr_img366_Rattenkampffest_in_London.jpg/198px-Das_festliche_Jahr_img366_Rattenkampffest_in_London.jpg)

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import folium
from folium import plugins

from IPython.display import HTML

# import plotly
import plotly.plotly as py
import plotly.graph_objs as go

# these two lines are what allow your code to show up in a notebook!
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode()

# read data
data = pd.read_csv("../input/311-service-requests-rodent-baiting.csv")

# Create DateTime index

# replace 'T' in a string with space character
datetime = data['Creation Date'].str.replace('T',' ')
# create a new column of 'creation_date' of datetime type
data['creation_date']=pd.to_datetime(datetime)
# set 'creation_date' as a new index
data.set_index('creation_date', inplace=True)
# drop the column 'Creation Date'
data.drop(axis='columns',inplace=True,columns=['Creation Date'])

# Save open requests to req_open
req_open = data[data.Status == 'Open'].copy()

# drop nulls
req_open.dropna(inplace=True, subset=['Latitude','Longitude'],axis=0)

# Filter recent actions, keep 'Inspected and baited' and 'Area inspected, no baiting, owner responsibility'
# Inspectors found rats or rat tracks or other signs of rats' activity in premises
data_baited=data[(data['Most Recent Action']=='Inspected and baited')|(data['Most Recent Action']=='Area inspected, no baiting, owner responsibility')]

# Take only completed, confirmed requests
data_baited = data_baited[data_baited.Status == 'Completed']

datefrom = data_baited.index.max()-pd.Timedelta(days=30)
data_30d = data_baited[data_baited.index>datefrom].copy()

# delete rows with coordinate columns not defined
data_30d.dropna(inplace=True, subset=['Latitude','Longitude'],axis=0)

# count daily requests
data_req_daily = data_baited.index.value_counts()
data_req_daily.name = 'requests_num'
# resample by month
data_req_monthly = data_req_daily.resample('M').mean().fillna(0).astype(int)
data_req_monthly_min = data_req_daily.resample('M').min().fillna(0).astype(int)
data_req_monthly_max = data_req_daily.resample('M').max().fillna(0).astype(int)


# display date range beginning from two years ago
range_start = data_req_monthly.index.max()-pd.Timedelta(days=2*365)
range_display = (range_start,data_req_monthly.index.max())


# sepcify that we want a scatter plot with, with date on the x axis and meet on the y axis
draw_data = [go.Scatter(x=data_req_monthly.index, y=data_req_monthly,
                        mode = 'lines+markers',
                        name = 'Avg',
                        line = dict(width=3),
                        marker = dict(size=8)),
             go.Scatter(x=data_req_monthly_min.index, y=data_req_monthly_min,
                        mode = 'lines+markers',
                        name = 'Min',
                        line = dict(dash = 'dash', color='rgb(115,115,115)')),
             go.Scatter(x=data_req_monthly_max.index, y=data_req_monthly_max,
                        mode = 'lines+markers',
                        name = 'Max',
                        line = dict(dash='dash',color ='rgb(189,189,189)'))

            ]

# specify the layout of our figure
layout = dict(title = "Daily Count of Confirmed Rodent Baiting Requests by Month",
              xaxis= dict(title= 'Date',ticklen= 2,zeroline= False,range = range_display)
              #,width = 800
             )

# create and show our figure
fig = dict(data = draw_data, layout = layout)
#iplot(fig)

# Create Chicago map with open requests 
# Create `req_open_loc` by subsetting only Latitude and Longitude from the dataset 
req_open_loc = req_open[['Latitude','Longitude']]

# Create `req_open_list` by transforming the DataFrame to list of lists 
req_open_list = req_open_loc.values.tolist()

# Check the length of the list
#len(req_open_list)


# Create a Map object, location and zoom
center_latitude, center_longitude = 41.881832,  -87.623177

location = (center_latitude, center_longitude)
zoom_start = 11

chicago_open_req_map = folium.Map(location=location, zoom_start=zoom_start
                         ,control_scale=True,tiles='stamentoner'
                         #,width=800 
                         #,height=600 
                        )

# Add open requests to the map
for point in range(0, len(req_open_list)):
    folium.CircleMarker(req_open_list[point], radius=2, color=None, fill=True, fill_color='red', fill_opacity = 0.5).add_to(chicago_open_req_map)


# Create the HeatMap layer
radius_pixels = 15 
blur = 10
gradient = None 
heatmap = plugins.HeatMap(req_open_list,name = "Heatmap of open requests", radius=radius_pixels, blur=blur, gradient=gradient)
# Add Heatmap Layer to the map
heatmap.add_to(chicago_open_req_map) # or map_.add_child(heatmap)

# create an HTML
chicago_open_req_map.save('chicago_open_req_map.html')

#Display in notebook
#chicago_map


#Create Chicago map with completed request for the last 30 days 
# Create `data_30d_loc` by subsetting only Latitude and Longitude from the dataset 
data_30d_loc = data_30d[['Latitude','Longitude']]

# Create `data_30d_list` by transforming the DataFrame to list of lists 
data_30d_list = data_30d_loc.values.tolist()

chicago_data_30d_map = folium.Map(location=location, zoom_start=zoom_start
                         ,control_scale=True,tiles='stamentoner'
                         #,width=800 
                         #,height=600 
                        )

# Create the HeatMap layer
radius_pixels = 15 
blur = 10
gradient = None 

heatmap = plugins.HeatMap(data_30d_list,name = "Heatmap of Completed Requests for the Last 30 Days", radius=radius_pixels, blur=blur, gradient=gradient)

# Add Heatmap Layer to the map
heatmap.add_to(chicago_data_30d_map) # or map_.add_child(heatmap)

# create an HTML
chicago_data_30d_map.save('chicago_data_30d_map.html')

# Create Chicago animated map with completed requests for the last 30 days
# sort rows by dates 
data_30d.sort_index(inplace=True)

# index_steps - steps of visualization
index_steps = data_30d.index.unique().astype(str).tolist()

# l_l_points is the list of lists of loctions
l_l_points = [] # needs to be the same size as index_steps
for d in data_30d.index.unique():
    data_30d_loc = data_30d.loc[data_30d.index == d,['Latitude','Longitude']]
    data_30d_list = data_30d_loc.values.tolist()
    l_l_points.append(data_30d_list)

chicago_30d_ani_map = folium.Map(location=location, zoom_start=zoom_start
                            ,control_scale=True
                            ,tiles='stamentoner'
                            #,width=800 
                           )

# Create the HeatMapWithTime layer
auto_play = True # means that the animation will start automatcally
heatmap_wTime = plugins.HeatMapWithTime(l_l_points, index_steps
                                        ,auto_play=auto_play, display_index=True
                                        ,radius=radius_pixels, gradient=gradient
                                        ,position='topright'
                                       )

# Add Marker Group Layer to the map
#heatmap_wTime.add_to(chicago_map_30d)
heatmap_wTime.add_to(chicago_30d_ani_map)

# Create HTML
chicago_30d_ani_map.save('chicago_30d_ani_map.html')

# display in notebook    
#chicago_30d_ani_map

# Display plots
iplot(fig)

## Open Requests

In [ ]:
HTML('<iframe src=chicago_open_req_map.html height=600 width="100%" frameBorder="0"></iframe>')

## Completed Requests (last 30 days)

In [ ]:
HTML('<iframe src=chicago_data_30d_map.html height=600 width="100%" frameBorder="0"></iframe>')

## Completed Requests  Day-by-Day (last 30 days)

In [ ]:
HTML('<iframe src=chicago_30d_ani_map.html  height=600 width="100%" frameBorder="0"></iframe>')
#HTML('<iframe src=plot_data.html width=700 height=450></iframe>')
